# Analise dos dados da PED

Jupyter Notebook para analisar os dados da Pesquisa de Emprego e Desemprego conforme estão em `raw/DIEPS/bases/ped-parquet/NovaPEDDF2018.parquet`.

## Leitura do MinIO

Leitura de todos os arquivos carregados do MinIO.

In [2]:
import pyarrow.dataset as pds
import pyarrow.parquet as pq
import pyarrow as pa
import s3fs
import pandas as pd

In [3]:
ACCESS_KEY = ''
SECRET_KEY = ''
MINIO_HOST = 'http://minio:9000'

In [4]:
minio = s3fs.S3FileSystem(key=ACCESS_KEY, # change default key 
                        secret=SECRET_KEY, # change default secret
                        use_ssl=False,
                        client_kwargs={
                            'endpoint_url': MINIO_HOST} # change ip
                        )

# Verifiacação da conexão 
objects = minio.head('raw/DIEPS/bases/nova-ped-parquet')
objects

## Carregamento em dataframe

Os arquivos parquet listados acima serão carregados em um único dataframe.

In [106]:
from io import BytesIO
fs = s3fs.core.S3FileSystem()

part = pds.partitioning(
    pa.schema([
        ('year', pa.int16())
    ])
)
dataset = pq.ParquetDataset(
    'raw/DIEPS/bases/nova-ped-parquet',
    filesystem=minio,
    partitioning=part
)


# dt = dataset.to_table()
# dt
dataset.files

In [109]:
fields = []
for field in dataset.schema:
    fields.append(field.name)
#del fields[83]
fields

In [111]:
# # Converte para pandas
fields_final = fields[0:82] + fields[83:]
# len(fields_final)

#df = dataset.read(columns=fields_final).to_pandas()
import re

# Set the columns list to all fields that match the regex r'(C010)|F[0-9]*'
#fields = [field for field in fields_final if re.match(r'(C030)|(F[0-9]*)|(year)', field)]
#fields = [field for field in fields_final if re.match(r'(C030)|(SIT)|(year)', field)]
#fields = fields[['C']]

df = dataset.read(columns=fields_final).to_pandas()

df[['C030', 'F010', 'F030', 'F070',  'year']]

#df[(df['year'] == 2018) & (df['SIT'] != 'Ocupado')]

In [121]:
df2 = df[df['year'] == 2022].copy()

In [122]:
# desempregado = list(df2['SIT'].cat.categories)
# desempregado

In [ ]:
#pd.set_option('display.max_rows', None)

# População economicamente ativa (PEA)
pea = df2[
    (
        (df['SIT'] != 'Inativo com Trabalho Excepcional') &
        (df['SIT'] != 'Inativo Puro') &
        (df['SIT'] != 'Não se aplica (menor de 14 anos)')
    ) &
    (
        (df2['F010'] == 'Sim') | # Procurou trabalho nos últimos 30 dias?
        (df2['F030'] == 'Sim') | # Procurou trabalho nos últimos 12 meses?
        (df2['F070'] != 'Não se aplica') # Fez trabalho nos últimos 7 dias?
    )
].groupby(['C030'])['C030'].count()
pea

In [ ]:
# Desempregados
pea_desempregados = df2[
    (
        (df2['SIT'] == 'Desemprego Oculto pelo Trabalho Precário') |
        (df2['SIT'] == 'Desemprego Aberto') |
        (df2['SIT'] == 'Desemprego Oculto pelo Desalento')
    ) &
    (
        (df2['F010'] == 'Sim') |
        (df2['F030'] == 'Sim') &
        (df2['F070'] == 'Não')
    )].groupby(['C030'])['C030'].count()
pea_desempregados

In [125]:
# taxa = (pea_desempregados / pea) * 100
# print(f'{taxa:.2f}%')
df_joined = pd.DataFrame(
    {
        'PEA': pea, 
        'PEA Desempregados': pea_desempregados
     }
)
df_joined
# pea

In [126]:
# Taxa de desemprego por sexo

(df_joined['PEA Desempregados'] / df_joined['PEA']) * 100

# Analise

Gráficos para análise e possível identificação de indicadores nos dados

In [ ]:
# Converte os valores 1000000000 para NaN
import numpy as np
df2 = df.copy()

Quantidade de pessoas que receberam o bolsa família por sexo e cor entre 2016 e 2023.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

# Cores
pastel_colors = ['#98BFA5', '#F7CAC9', '#87CEEB', '#ADD8E6', '#D2B48C']

# Renomeia as colunas
df2.rename(columns={
    'C040': 'Cor',
    'C030': 'Sexo',
    'year': 'Ano'
}, inplace=True)

# Realiza o agupamento por cor e sexo
fig = plt.figure(dpi=100)
df2.groupby(['Cor','Sexo'])['D010'].agg('count').unstack('Sexo').plot.bar(color=pastel_colors, ax=plt.gca())

# Apresenta o gráfico
plt.title('Quantidade de pessoas que receberam Bolsa Família por sexo e cor')
plt.show()


Quantidade de pessoas que receberam bolsa família por cor, por ano.


In [ ]:
# Cores
pastel_colors = ['#98BFA5', '#F7CAC9', '#87CEEB', '#ADD8E6', '#D2B48C']

fig = plt.figure(dpi=100)
df2.groupby(['Cor','Ano'])['D010'].agg('count').unstack('Cor').plot.line(color=pastel_colors, ax=plt.gca())
plt.title('Quantidade de pessoas que receberam Bolsa Família por cor')
plt.show()
